In [222]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from collections import Counter
from random import randrange
from profanity_check import predict, predict_prob

In [5]:
songs = pd.read_csv('../data/songs_after_round_3_cleaning.csv')

# Build Lyrics Corpus

In [7]:
lyrics_corpus = ""

for l in list(songs['cleaned_lyrics']):
    lyrics_corpus += l

In [9]:
lyrics_list = lyrics_corpus.split()

# Get Probabilty Word Follows Another Word to Generate Lyrics

In [325]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(6)

In [326]:
dirty = ['fuck','ass','shit','bitch','cunt','nigga','nigger','asshole','penis','bitches']

In [339]:
def create_lyrics(start_word, length):
    
    new_word = start_word
    lyrics = start_word + ' '
    
    for i in range(length):
        
        following_words = []
        
        for i in range(len(lyrics_list)):
            if (lyrics_list[i] == new_word) and (i != len(lyrics_list) -1):
                following_words.append(lyrics_list[i+1])
    
        if len(set(following_words)) > 10:
            fol = most_frequent(following_words)[randrange(5)][0]
            if fol in dirty:
                fol = most_frequent(following_words)[randrange(5)][0]
            if fol in dirty:
                fol = most_frequent(following_words)[randrange(5)][0]
 
            lyrics += fol + " "
            new_word = fol
            
        else:
            if most_frequent(following_words)[0][0] in dirty:
                lyrics += '***censored***' + " "
                new_word = most_frequent(following_words)[0][0]
            else:
                lyrics += str(most_frequent(following_words)[0][0]) + " "
                new_word = most_frequent(following_words)[0][0]
            
        
    return(lyrics)

In [344]:
create_lyrics('people', 20)

'people say you dont want to you can be the one of the world we got it aint gotta keep it '

In [345]:
create_lyrics('wow', 20)

'wow and i just wanna know how we gon get to you can get you want a bad yeah [chorus] and '

In [346]:
create_lyrics('tonight', 10)

'tonight im just wanna be the same way that you know '

In [348]:
create_lyrics('bad', 20)

'bad for the world i cant be there is the way you dont know you can see it i know that '